Installation d'un package via pip
```bash
pip install --proxy=10.150.2.30:8080 wget
```

Création de la table de destination des requêtes Power BI
```sql
USE [Staging]
GO

IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'MDW.Dim_PowerBIRapportRequetes') AND type in (N'U'))
BEGIN
       DROP TABLE MDW.Dim_PowerBIRapportRequetes
END



CREATE TABLE MDW.Dim_PowerBIRapportRequetes(
	[Id Item] varcher(36) NOT NULL,
	[Libellé du rapport] [nvarchar](425) NOT NULL,
	[Path] [nvarchar](425) NOT NULL,
	[Requêtes] ntext NOT NULL,
	[Date chargement] [datetime2](0) NOT NULL CONSTRAINT DF_Dim_EvenementLocalisation_Date_chargement_GETDATE DEFAULT GETDATE(),
	[User chargement] [varchar](128) NOT NULL CONSTRAINT DF_Dim_EvenementLocalisation_User_chargement_SYSTEM_USER DEFAULT SYSTEM_USER
) ON [FG_MDW]

GO
```

In [27]:
import requests
import re
from requests_negotiate_sspi import HttpNegotiateAuth
import os
print(os.getcwd())
    
# Pour un rapport donné
url = 'http://sqlinfobi/ReportsPowerBi/api/v2.0/catalogitems(62035377-4c4a-4150-9813-87c4c06e4f0e)/Content/$value'

r = requests.get(url, allow_redirects=True, auth=HttpNegotiateAuth())
#r = requests.get(url, auth=HttpNegotiateAuth())
# retrieve filename
filename = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
# save file
#open(str(filename), 'wb').write(r.content)

open(str(filename), 'wb').write(r.content)


C:\Users\berhaultb\Documents\Python Scripts\Power BI


50800

In [18]:
url = 'http://sqlinfobi/ReportsPowerBi/api/v2.0/catalogitems(b0ae3ac6-76fb-4598-987b-968417cea48b)/Content/$value'
r = requests.get(url, allow_redirects=True, auth=HttpNegotiateAuth())

In [19]:
import requests
import os
import re
from requests_negotiate_sspi import HttpNegotiateAuth

subfolder = 'power_bi_reports'

if not os.path.exists(subfolder):
    os.makedirs(subfolder)
    
def download_pbix(ItemID, Path):
    print('Dans download_pbix, ItemID: '+ItemID+' ; Path: '+Path)

    url = 'http://sqlinfobi/ReportsPowerBi/api/v2.0/catalogitems('+ItemID+')/Content/$value'

    r = requests.get(url, allow_redirects=True, auth=HttpNegotiateAuth())
    print('Après requests.get')
    # r = requests.get(url, auth=HttpNegotiateAuth())
    # retrieve filename
    # filename = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
    # save file
    print(subfolder + Path.rsplit('/', 1)[0] + '/' + ItemID + '.zip')
    os.makedirs(subfolder+Path.rsplit('/', 1)[0], exist_ok=True)
    print(subfolder + Path + '.zip')

    #open(str(filename), 'wb').write(r.content)
    open(subfolder + Path + '.zip', 'wb').write(r.content)
    
    return str(subfolder + Path), str(subfolder + Path + '.zip')   
    

In [20]:
import pyodbc
from subprocess import check_output
    
cnxn = pyodbc.connect(r'Driver={SQL Server};Server=SQLINFOBI;Database=ReportServePowerbi;Trusted_Connection=yes;')
cursor = cnxn.cursor()
cursor.execute("SELECT ItemID, Path, Name \
  FROM [ReportServePowerbi].[dbo].[Catalog] t  \
  WHERE Type = 13")
"""
CASE Type
       WHEN 1 THEN 'Folder'
       WHEN 2 THEN 'Report Builder'
       WHEN 5 THEN 'Data Source'
       WHEN 7 THEN 'Report Part'
       WHEN 8 THEN 'Shared Dataset'
	   When 13 Then 'Power BI Report'
       ELSE 'Other'
     END AS TypeDescription
"""
while 1:
    row = cursor.fetchone()
    print('Avant if not row:')
    if not row:
        break
    print('Avant folder, file_zip')
    folder, file_zip = download_pbix(row.ItemID, row.Path)
    print('Après folder, file_zip')
    print(file_zip)
    print('"C:/Program Files (x86)/7-Zip/7z.exe" e "'+file_zip+'" "-o'+folder+'" DataMashup')
    # Extraction de DataMashup
    check_output('"C:/Program Files (x86)/7-Zip/7z.exe" e "'+file_zip+'" "-o'+folder+'" DataMashup -y', shell=True,encoding="437")
    # Extraction de Section1.m
    check_output('"C:/Program Files (x86)/7-Zip/7z.exe" e "'+folder+'/DataMashup" "-o'+folder+'" Section1.m -r -y', shell=True,encoding="437")
    # Suppression de DataMashup
    check_output('del "'+folder.replace("/", "\\")+'\DataMashup"', shell=True,encoding="437")
    f = open(folder+'/Section1.m', 'r')
    #contenu du fichier
    file_contents = f.read()
    """
    [Id Item] : row.ItemID
	[Libellé du rapport] : 
	[Path] : folder,
	[Requêtes] : file_contents
    """
    print(file_contents)
cnxn.close()

Avant if not row:
Avant folder, file_zip
Dans download_pbix, ItemID: B0AE3AC6-76FB-4598-987B-968417CEA48B ; Path: /01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/Bellaud/ZRapport1_PSE_VSAV_Asso
Après requests.ge
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/Bellaud/B0AE3AC6-76FB-4598-987B-968417CEA48B.zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/Bellaud/ZRapport1_PSE_VSAV_Asso.zip
Après folder, file_zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/Bellaud/ZRapport1_PSE_VSAV_Asso.zip
"C:/Program Files (x86)/7-Zip/7z.exe" e "power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/Bellaud/ZRapport1_PSE_VSAV_Asso.zip" "-opower_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/Bellaud/ZRapport1_PSE_VSAV_Asso" DataMashup
section Section1;

shared #"SID_OPE VentilationMinute" = let
    Source = Sql.Databases("DESKTOP-5762U75"),
    Entrepot 

Après requests.ge
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/99270DD1-C9E2-4A41-8923-D95E808A6721.zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_debut.zip
Après folder, file_zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_debut.zip
"C:/Program Files (x86)/7-Zip/7z.exe" e "power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_debut.zip" "-opower_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_debut" DataMashup
section Section1;

shared #"DÃ©parts BSPP" = let
    Source = Table.Buffer(Excel.Workbook(File.Contents("\\bspp.fr\Travail\BOSI\STOU\Partage\Section RÃ©alisation des SI\Groupe Info DÃ©cisionnelle\DonnÃ©es externes PowerBi.xlsx"), null, true)),
    #"DÃ©parts BSPP_Sheet" = Source{[I

Après requests.ge
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/364C4A81-8739-44FA-B21C-9C2AAF5661ED.zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_fin.zip
Après folder, file_zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_fin.zip
"C:/Program Files (x86)/7-Zip/7z.exe" e "power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_fin.zip" "-opower_bi_reports/01 - Principal/01 - Rapports Power BI/dev/BOSI/Infodecision/BerhaultB/Infocentre_table_buffer_en_fin" DataMashup
section Section1;

shared #"DÃ©parts BSPP" = let
    Source = Table.Buffer(Excel.Workbook(File.Contents("\\bspp.fr\Travail\BOSI\STOU\Partage\Section RÃ©alisation des SI\Groupe Info DÃ©cisionnelle\DonnÃ©es externes PowerBi.xlsx"), null, true)),
    #"DÃ©parts BSPP_Sheet" = Source{[Item="DÃ©

Après requests.ge
power_bi_reports/01 - principal/01 - rapports power bi/dev/bosi/infodecision/berhaultb/DDDA95A4-02CF-4232-B073-FBFDCC039419.zip
power_bi_reports/01 - principal/01 - rapports power bi/dev/bosi/infodecision/berhaultb/interventions bspp cma cri départements.zip
Après folder, file_zip
power_bi_reports/01 - principal/01 - rapports power bi/dev/bosi/infodecision/berhaultb/interventions bspp cma cri départements.zip
"C:/Program Files (x86)/7-Zip/7z.exe" e "power_bi_reports/01 - principal/01 - rapports power bi/dev/bosi/infodecision/berhaultb/interventions bspp cma cri départements.zip" "-opower_bi_reports/01 - principal/01 - rapports power bi/dev/bosi/infodecision/berhaultb/interventions bspp cma cri départements" DataMashup
section Section1;

shared #"SQLINFOSERVICE Intervention" = let
    Source = Sql.Database("SQLINFOSERVICE", "Entrepot", [Query="
SELECT
      [Date dÃ©crochÃ©]
      ,[IdDate dÃ©crochÃ©]
      ,[IdTime dÃ©crochÃ©]
	  ,[IdTime dÃ©crochÃ©] / 10000 As 'Heure

section Section1;

shared BI_Dim_Date = let
    Source = Sql.Database("SQLBI2014", "Entrepot"),
    dbo_BI_Dim_Date = Source{[Schema="dbo",Item="BI_Dim_Date"]}[Data],
    #"Lignes filtrÃ©es" = Table.SelectRows(dbo_BI_Dim_Date, each [idDate] >= 20160101 and [idDate] <= 20190101)
in
    #"Lignes filtrÃ©es";

shared BI_Dim_Time = let
    Source = Sql.Database("SQLBI2014", "Entrepot"),
    dbo_BI_Dim_Time = Source{[Schema="dbo",Item="BI_Dim_Time"]}[Data]
in
    dbo_BI_Dim_Time;

shared #"TPH Dim_CallDepartment" = let
    Source = Sql.Database("SQLBI2014", "Entrepot"),
    TPH_Dim_CallDepartment = Source{[Schema="TPH",Item="Dim_CallDepartment"]}[Data]
in
    TPH_Dim_CallDepartment;

shared #"TPH Dim_CallFlow" = let
    Source = Sql.Database("SQLBI2014", "Entrepot"),
    TPH_Dim_CallFlow = Source{[Schema="TPH",Item="Dim_CallFlow"]}[Data]
in
    TPH_Dim_CallFlow;

shared #"TPH Dim_CallDomain" = let
    Source = Sql.Database("SQLBI2014", "Entrepot"),
    TPH_Dim_CallDomain = Source{[Schema="TP

...

Après requests.ge
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/1F338FDF-F402-455B-9E5E-C0B6A70AD20B.zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/brq bspp corrigé.zip
Après folder, file_zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/brq bspp corrigé.zip
"C:/Program Files (x86)/7-Zip/7z.exe" e "power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/brq bspp corrigé.zip" "-opower_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/brq bspp corrigé" DataMashup
section Section1;

shared #"Info_Centre Compte Rendu Intervention" = let
    Source = Sql.Database("SQLCHPT\IC", "ADAGIO", [Query="SELECT cri.IdCRI,
	cri.CodeCRI + '. ' + cri.LibelleCRI As 'LibellÃ© CRI',
	SUBSTRING(cri_parent.CodeCRI, 1, 1) As 'Code catÃ©gorie CRI',
	SUBSTRING(cri_parent.CodeCRI, 1, 1) + '. ' + cri_parent.LibelleCRI As 'LibellÃ© catÃ©gorie CRI'
FROM Rapport.R_CRI_New cri WITH (NOLOCK) 
	INNER JO

Après requests.ge
power_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/9DA1BC0A-C670-4D0A-83BA-4C1567D1E914.zip
power_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/brq bspp v2.2.4.zip
Après folder, file_zip
power_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/brq bspp v2.2.4.zip
"C:/Program Files (x86)/7-Zip/7z.exe" e "power_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/brq bspp v2.2.4.zip" "-opower_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/brq bspp v2.2.4" DataMashup
section Section1;

shared #"Info_Centre Compte Rendu Intervention" = let
    Source = Sql.Database("SQLCHPT\IC", "ADAGIO", [Query="SELECT cri.IdCRI,
	cri.CodeCRI + '. ' + cri.LibelleCRI As 'LibellÃ© CRI',
	SUBSTRING(cri_parent.CodeCRI, 1, 1) As 'Code catÃ©gorie CRI',
	SUBSTRING(cri_parent.CodeCRI, 1, 1) + '. ' + cri_parent.LibelleCRI As 'LibellÃ© catÃ©gorie CRI'
FROM Rapport.R_CRI_New cri WITH (NOLOCK) 
	INNER JOIN (

Après requests.ge
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/B7D05F86-7299-4B0F-992C-E0953422B294.zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/brq coordination médicale.zip
Après folder, file_zip
power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/brq coordination médicale.zip
"C:/Program Files (x86)/7-Zip/7z.exe" e "power_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/brq coordination médicale.zip" "-opower_bi_reports/01 - Principal/01 - Rapports Power BI/dev/siop/vorières/brq coordination médicale" DataMashup
section Section1;

shared #"TPH Fait_Appel" = let
    Source = Sql.Databases("sqlinfoservice"),
    Entrepot = Source{[Name="Entrepot"]}[Data],
    TPH_Fait_Appel = Entrepot{[Schema="TPH",Item="Fait_Appel"]}[Data],
    #"Lignes filtrÃ©es" = Table.SelectRows(TPH_Fait_Appel, each [Connexion CM Ã©tablie] = true),
    #"TPH.Dim_CallFlow dÃ©veloppÃ©" = Table.ExpandRecordColumn(#"Lig

Après requests.ge
power_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/4A1FD607-77F1-448C-91C0-CB0B3080917D.zip
power_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/brq gis 1.zip
Après folder, file_zip
power_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/brq gis 1.zip
"C:/Program Files (x86)/7-Zip/7z.exe" e "power_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/brq gis 1.zip" "-opower_bi_reports/01 - principal/01 - rapports power bi/dev/siop/vorières/brq gis 1" DataMashup


KeyboardInterrupt: 

In [ ]:
import pyodbc 

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=SQLINFOSERVICE;"
                      "Database=Staging;"
                      "Trusted_Connection=yes;")

cursor = cnxn.cursor()
cursor.execute('TRUNCATE TABLE Staging.MDW.Dim_PowerBIRapportRequetes; \
insert into Staging.MDW.Dim_PowerBIRapportRequetes([Id Item], [Libellé du rapport] ,[Path], [Requêtes])  \
values();'\)

In [151]:
from zipfile import ZipFile
import csv


with ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip') as myzip:
    with myzip.open('DataMashup/Formulas/Section1.m') as myfile:
        mc = myfile.read()
        c = csv.StringIO(mc.decode())
        for row in c:
            print(row)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfd in position 20: invalid start byte

In [156]:
import zipfile
with zipfile.ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip', 'r') as zip_ref:
    zip_ref.extractall('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac')

In [158]:
f = open('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac/DataMashup/Formulas/Section1.m', 'r')
file_contents = f.read()
print (file_contents)

FileNotFoundError: [Errno 2] No such file or directory: 'power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac/DataMashup/Formulas/Section1.m'

In [165]:
import os
import shutil

with zipfile.ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip') as z:
    with z.open('/DataMashup/[Content_Types].xml') as zf, open('[Content_Types].xml', 'wb') as f:
        shutil.copyfileobj(zf, f)

KeyError: "There is no item named '/DataMashup/[Content_Types].xml' in the archive"

In [168]:
import os

with zipfile.ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip') as z:
    with open('[Content_Types].xml', 'wb') as f:
        f.write(z.read('DataMashup'))

In [177]:
import zipfile
import io
with zipfile.ZipFile('power_bi_reports/01 - Principal/01 - Rapports Power BI/public/Téléphonie/pfau bpac.zip') as z:
    with z.open('DataMashup') as z2:
        z2_filedata =  io.BytesIO(z2.read())
        with zipfile.ZipFile(z2_filedata) as nested_zip:
            print( nested_zip.open('Formulas/Section1.m').read())
         

KeyError: "There is no item named 'Formulas/Section1.m' in the archive"

In [61]:
import os
for dirpath, dirnames, filenames in os.walk('.'):
    for f in filenames:
        print(os.path.join(dirpath, f))

.\Récupération des rapports Power BI.ipynb
.\temp.pbix
.\voila
.\.ipynb_checkpoints\Récupération des rapports Power BI-checkpoint.ipynb


In [22]:
import requests

def is_downloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True

print(is_downloadable(url))

AttributeError: 'NoneType' object has no attribute 'lower'

In [23]:
import requests
resp = requests.get(url)

In [25]:
resp.content

b''

In [65]:
import requests
req = requests.get(url)
req

<Response [401]>

In [28]:
import urllib, urlparse

split = urlparse.urlsplit(url)
filename = "C:/Users/berhaultb/Documents/Python Scripts/Power BI" + split.path.split("/")[-1]
urllib.urlretrieve(url, filename)

ModuleNotFoundError: No module named 'urlparse'

In [30]:
from urllib2 import urlopen

f = urlopen(url)
for line in f:
  print len(line)
f.close()

SyntaxError: invalid syntax (<ipython-input-30-391f44a24fd0>, line 5)